# Importing necessary libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import numpy as np

# CP method metrices(specific to Cervical Cancer Dataset):- 

In [3]:
def avg_set_size_metric(conformal_set):
    lengths = torch.sum(conformal_set, dim=1)
    avg_set_size_len = torch.sum(lengths)/conformal_set.shape[0]
    return avg_set_size_len




def coverage_gap_metric(conformal_set, df_true_class_test, alpha):
    true_class = conformal_set[range(conformal_set.shape[0]), df_true_class_test]
    tensor_sum = torch.sum(true_class)
    coverage = tensor_sum/true_class.shape[0]
    coverage_gap = (abs((1-alpha) - coverage)/(1-alpha))*100
    return coverage_gap, coverage





def class_Overlap_metric(conformal_set, label):

    overlap_count = 0

    for i in range(conformal_set.shape[0]):
        current_set = conformal_set[i]

        if (label[i] == 0 or label[i] == 1 or label[i] == 2) and (current_set[4] == 1 or current_set[6] == 1 or current_set[7] == 1 or current_set[3] == 1 or current_set[5] == 1):
                overlap_count += 1
            
        elif (label[i] == 4 or label[i] == 6 or  label[i] == 7) and (current_set[0] == 1 or current_set[1] == 1 or current_set[2] or current_set[3] == 1 or current_set[5] == 1):
                overlap_count += 1

        elif (label[i] == 3 or label[i] == 5) and (current_set[4] == 1 or current_set[6] == 1 or current_set[7] == 1 or current_set[0] == 1 or current_set[1] == 1 or current_set[2] == 1):
                overlap_count += 1


    perecentage_of_overlap  = (overlap_count/conformal_set.shape[0])*100

    return perecentage_of_overlap






def confusion_set_Overlap_metric(conformal_set, label):

    overlap_count = 0

    for i in range(conformal_set.shape[0]):
        current_set = conformal_set[i]

        if (label[i] == 4) and (current_set[6] == 1):
            overlap_count += 1

        elif (label[i] == 6) and (current_set[4] == 1):
            overlap_count += 1

        
        elif (label[i] == 3) and (current_set[5] == 1):
            overlap_count += 1

        elif (label[i] == 5) and (current_set[3] == 1):
            overlap_count += 1

    perecentage_of_overlap  = (overlap_count/conformal_set.shape[0])*100

    return perecentage_of_overlap

# RAPS class :- 

In [4]:
class RAPS():
    def __init__(self, softmax, true_class, alpha, k_reg, lambd, rand=True):
        self.prob_output = softmax
        self.true_class = true_class
        self.alpha = (1 - alpha) * (1 + (1 / softmax.shape[0]))
        self.k_reg = k_reg
        self.lambd = lambd
        self.rand = rand
        
        
        
    def conformal_score(self):
        conformal_score = []
        for i in range(self.prob_output.shape[0]):
            true_class_prob = self.prob_output[i][self.true_class[i]]
            current_class_prob = self.prob_output[i]
            sorted_class_prob, _ = torch.sort(current_class_prob, descending=True)
            index = torch.nonzero(sorted_class_prob == true_class_prob).item()
            cumulative_sum = torch.sum(sorted_class_prob[:index + 1])
            
            if index - self.k_reg > 0:
               cumulative_sum = cumulative_sum + self.lambd*(index - self.k_reg)
            
            if self.rand:
                U = torch.rand(1).item()
                cumulative_sum = cumulative_sum - U*sorted_class_prob[index]

            conformal_score.append(cumulative_sum)
        
        conformal_score = torch.tensor(conformal_score)
        
        return conformal_score
    
    
    
    def quantile(self):
        conformal_scores = self.conformal_score()
        quantile_value = torch.quantile(conformal_scores, self.alpha)
        return quantile_value
    
    
    
    def prediction(self, softmax, quantile_value):
        prob_output = softmax
        prediction = torch.zeros(prob_output.shape[0], prob_output.shape[1])
        for i in range(prob_output.shape[0]):
            current_class_prob = prob_output[i]
            sorted_class_prob, _ = torch.sort(current_class_prob, descending=True)
            sum = 0
            j = 0
            for idx in range(len(sorted_class_prob)):
                if sum <= quantile_value:
                    sum += sorted_class_prob[idx]
                    if idx - self.k_reg > 0:
                        sum = sum + self.lambd*(idx - self.k_reg)
                    j += 1
                else:
                    break
                    
            """
            
            if self.rand:
                U = torch.rand(1).item()
                if j != prob_output.shape[1]:
                    N = torch.sum(sorted_class_prob[:j + 1]) - quantile_value
                else:
                    N = torch.sum(sorted_class_prob[:j]) - quantile_value
                if idx - self.k_reg > 0:
                    N += self.lambd*(j - self.k_reg)

                if j != prob_output.shape[1]:
                    D = sorted_class_prob[j]
                else:
                    D = sorted_class_prob[j-1]
                if idx - self.k_reg > 0:
                    D += self.lambd

                if N/D <= U:
                    j = j -1
                    
                """ 
        
            
            
            
                    
            for idx in range(j):
                index = torch.nonzero(current_class_prob == sorted_class_prob[idx]).item()
                prediction[i][index] = 1.0
                
        return prediction

# Main function :- 

In [35]:

def main(expt_no, Trials, alpha,k_reg, lambd, rand):
    avg_set_size_len_for_T_trials = []
    avg_coverage_gap_for_T_trials = []
    avg_coverage_for_T_trials = []
    
    normal_avg_set_size_len_for_T_trials = []
    normal_avg_coverage_gap_for_T_trials = []
    normal_avg_coverage_for_T_trials = []

    LG_avg_set_size_len_for_T_trials = []
    LG_avg_coverage_gap_for_T_trials = []
    LG_avg_coverage_for_T_trials = []

    HG_avg_set_size_len_for_T_trials = []
    HG_avg_coverage_gap_for_T_trials = []
    HG_avg_coverage_for_T_trials = []
    
    perecentage_of_overlap_for_T_trials = []

    confusion_set_Overlap_metric_for_T_trials = []
    
    
        
    for t in range(Trials):
        """
        
        print()
        print(f'Trials :- {t}')
        print()
        """
        
        
        # loading the annotation file :-
        #Expt1:-
        #path = ''
        #df = pd.read_csv(path)
        
        #Expt2:-
        #path = ''
        #df = pd.read_csv(path)
        
        #Expt3:-
        #path = ''
        #df = pd.read_csv(path)
        
        #Expt4:-
        #path = ''
        #df = pd.read_csv(path)
        
        
        df = df.sample(frac=1).reset_index(drop=True)
        
        # calib-test split :- 
        test_0, calib_0, test_1, calib_1, test_2, calib_2, test_3, calib_3, test_4, calib_4, test_5, calib_5, test_6, calib_6, test_7, calib_7, test_label, calib_label = train_test_split(df['0'],df['1'], df['2'],df['3'], df['4'],df['5'], df['6'],df['7'], df['Label'], test_size = 0.1, stratify=df['Label'], random_state=42)

        test_0 = test_0.to_frame()
        test_1 = test_1.to_frame()
        test_2 = test_2.to_frame()
        test_3 = test_3.to_frame()
        test_4 = test_4.to_frame()
        test_5 = test_5.to_frame()
        test_6 = test_6.to_frame()
        test_7 = test_7.to_frame()
        test = test_0.join(test_1, how='inner').join(test_2, how='inner').join(test_3, how='inner').join(test_4, how='inner').join(test_5, how='inner').join(test_6, how='inner').join(test_7, how='inner').join(test_label, how='inner')
        test = test.reset_index(drop=True)

        calib_0 = calib_0.to_frame()
        calib_1 = calib_1.to_frame()
        calib_2 = calib_2.to_frame()
        calib_3 = calib_3.to_frame()
        calib_4 = calib_4.to_frame()
        calib_5 = calib_5.to_frame()
        calib_6 = calib_6.to_frame()
        calib_7 = calib_7.to_frame()
        calib = calib_0.join(calib_1, how='inner').join(calib_2, how='inner').join(calib_3, how='inner').join(calib_4, how='inner').join(calib_5, how='inner').join(calib_6, how='inner').join(calib_7, how='inner').join(calib_label, how='inner')
        calib = calib.reset_index(drop=True)

        prob_output = calib.iloc[:,:-1]
        df_np = prob_output.values 
        df_prob_output_calib = torch.tensor(df_np, dtype=torch.float32)

        prob_output = test.iloc[:,:-1]
        df_np = prob_output.values
        df_prob_output_test = torch.tensor(df_np, dtype=torch.float32)
        
        true_class = calib.iloc[:,-1]
        df_np = true_class.values
        df_true_class_calib = torch.tensor(df_np, dtype=torch.int)


        true_class = test.iloc[:,-1]
        df_np = true_class.values
        df_true_class_test = torch.tensor(df_np, dtype=torch.int)
        
        
        
        conformal_wrapper = RAPS(df_prob_output_calib, df_true_class_calib, alpha, k_reg, lambd, rand)
        quantile_value = conformal_wrapper.quantile()
        #print(f'quantile_value :- {quantile_value}')
        
        conformal_set = conformal_wrapper.prediction(df_prob_output_test, quantile_value)
        
        
        if expt_no == 1:
            avg_set_size = avg_set_size_metric(conformal_set)
            coverage_gap, coverage = coverage_gap_metric(conformal_set, df_true_class_test, alpha)    
            avg_set_size_len_for_T_trials.append(avg_set_size)
            avg_coverage_gap_for_T_trials.append(coverage_gap)
            avg_coverage_for_T_trials.append(coverage)
            
            
            
        elif expt_no == 2:
            label = df_true_class_test
            indices_0 = torch.nonzero(label == 0).squeeze()
            indices_1 = torch.nonzero(label == 1).squeeze()
            indices_2 = torch.nonzero(label == 2).squeeze()
            indices_3 = torch.nonzero(label == 3).squeeze()
            indices_4 = torch.nonzero(label == 4).squeeze()
            indices_5 = torch.nonzero(label == 5).squeeze()
            indices_6 = torch.nonzero(label == 6).squeeze()
            indices_7 = torch.nonzero(label == 7).squeeze()


            Normal_idx = torch.cat((indices_0, indices_1, indices_2))
            LG_idx = torch.cat((indices_3, indices_5))
            HG_idx = torch.cat((indices_4, indices_6, indices_7))

            normal_conformal_prediction_set = conformal_set[Normal_idx, :]
            LG_conformal_prediction_set = conformal_set[LG_idx, :]
            HG_conformal_prediction_set = conformal_set[HG_idx, :]


            normal_avg_set_size_len = avg_set_size_metric(normal_conformal_prediction_set)
            LG_avg_set_size_len = avg_set_size_metric(LG_conformal_prediction_set)
            HG_avg_set_size_len = avg_set_size_metric(HG_conformal_prediction_set)
            
            normal_true_class = df_true_class_test[Normal_idx]
            LG_true_class = df_true_class_test[LG_idx]
            HG_true_class = df_true_class_test[HG_idx]


            normal_coverage_gap, normal_coverage = coverage_gap_metric(normal_conformal_prediction_set, normal_true_class, alpha)
            LG_coverage_gap, LG_coverage = coverage_gap_metric(LG_conformal_prediction_set, LG_true_class, alpha)
            HG_coverage_gap, HG_coverage = coverage_gap_metric(HG_conformal_prediction_set, HG_true_class, alpha)
            
            
            normal_avg_set_size_len_for_T_trials.append(normal_avg_set_size_len)
            normal_avg_coverage_gap_for_T_trials.append(normal_coverage_gap)
            normal_avg_coverage_for_T_trials.append(normal_coverage)

            LG_avg_set_size_len_for_T_trials.append(LG_avg_set_size_len)
            LG_avg_coverage_gap_for_T_trials.append(LG_coverage_gap)
            LG_avg_coverage_for_T_trials.append(LG_coverage)

            HG_avg_set_size_len_for_T_trials.append(HG_avg_set_size_len)
            HG_avg_coverage_gap_for_T_trials.append(HG_coverage_gap)
            HG_avg_coverage_for_T_trials.append(HG_coverage)
            
        
        elif expt_no == 3:
            perecentage_of_overlap = class_Overlap_metric(conformal_set, df_true_class_test)
            perecentage_of_overlap_for_T_trials.append(perecentage_of_overlap)
            
            
        elif expt_no == 4:
            perecentage_of_confusion = confusion_set_Overlap_metric(conformal_set, df_true_class_test)
            confusion_set_Overlap_metric_for_T_trials.append(perecentage_of_confusion)
            
            
            
            
            
            
        
        
        
        
        
        
        
        
        
        
        
        
        
        
    if expt_no == 1:       
        avg_set_size_len_for_T_trials = np.array(avg_set_size_len_for_T_trials)
        average_len = np.mean(avg_set_size_len_for_T_trials)
        std_dev_len = np.std(avg_set_size_len_for_T_trials, ddof=1)



        avg_coverage_gap_for_T_trials = np.array(avg_coverage_gap_for_T_trials)
        average_coverage_gap = np.mean(avg_coverage_gap_for_T_trials)
        std_dev_coverage_gap = np.std(avg_coverage_gap_for_T_trials, ddof=1)



        avg_coverage_for_T_trials = np.array(avg_coverage_for_T_trials)
        average_coverage = np.mean(avg_coverage_for_T_trials)
        std_dev_coverage = np.std(avg_coverage_for_T_trials, ddof=1)
        
        
        return average_len, std_dev_len, average_coverage_gap, std_dev_coverage_gap, average_coverage, std_dev_coverage
        
        
        
        
        
    elif expt_no == 2:
        
        #set_size:- 
        normal_avg_set_size_len_for_T_trials = np.array(normal_avg_set_size_len_for_T_trials)
        normal_average_set_size_len = np.mean(normal_avg_set_size_len_for_T_trials)
        normal_std_dev_set_size_len = np.std(normal_avg_set_size_len_for_T_trials, ddof=1)
        
        LG_avg_set_size_len_for_T_trials = np.array(LG_avg_set_size_len_for_T_trials)
        LG_average_set_size_len = np.mean(LG_avg_set_size_len_for_T_trials)
        LG_std_dev_set_size_len = np.std(LG_avg_set_size_len_for_T_trials, ddof=1)
        
        HG_avg_set_size_len_for_T_trials = np.array(HG_avg_set_size_len_for_T_trials)
        HG_average_set_size_len = np.mean(HG_avg_set_size_len_for_T_trials)
        HG_std_dev_set_size_len = np.std(HG_avg_set_size_len_for_T_trials, ddof=1)
        
        
        
        #coverage_gap:-
        normal_avg_coverage_gap_for_T_trials = np.array(normal_avg_coverage_gap_for_T_trials)
        normal_avg_coverage_gap = np.mean(normal_avg_coverage_gap_for_T_trials)
        normal_std_dev_coverage_gap = np.std(normal_avg_coverage_gap_for_T_trials)
        
        LG_avg_coverage_gap_for_T_trials = np.array(LG_avg_coverage_gap_for_T_trials)
        LG_avg_coverage_gap = np.mean(LG_avg_coverage_gap_for_T_trials)
        LG_std_dev_coverage_gap = np.std(LG_avg_coverage_gap_for_T_trials)
        
        HG_avg_coverage_gap_for_T_trials = np.array(HG_avg_coverage_gap_for_T_trials)
        HG_avg_coverage_gap = np.mean(HG_avg_coverage_gap_for_T_trials)
        HG_std_dev_coverage_gap = np.std(HG_avg_coverage_gap_for_T_trials)
        
        
        
        # coverage:-
        
        normal_avg_coverage_for_T_trials = np.array(normal_avg_coverage_for_T_trials)
        normal_average_coverage = np.mean(normal_avg_coverage_for_T_trials)
        normal_std_dev_coverage = np.std(normal_avg_coverage_for_T_trials, ddof=1)
        
        LG_avg_coverage_for_T_trials = np.array(LG_avg_coverage_for_T_trials)
        LG_average_coverage = np.mean(LG_avg_coverage_for_T_trials)
        LG_std_dev_coverage = np.std(LG_avg_coverage_for_T_trials, ddof=1)
        
        HG_avg_coverage_for_T_trials = np.array(HG_avg_coverage_for_T_trials)
        HG_average_coverage = np.mean(HG_avg_coverage_for_T_trials)
        HG_std_dev_coverage = np.std(HG_avg_coverage_for_T_trials, ddof=1)
        
        
        return normal_average_set_size_len, normal_std_dev_set_size_len, LG_average_set_size_len, LG_std_dev_set_size_len, HG_average_set_size_len, HG_std_dev_set_size_len, normal_avg_coverage_gap, normal_std_dev_coverage_gap, LG_avg_coverage_gap, LG_std_dev_coverage_gap, HG_avg_coverage_gap, HG_std_dev_coverage_gap, normal_average_coverage, normal_std_dev_coverage, LG_average_coverage, LG_std_dev_coverage, HG_average_coverage, HG_std_dev_coverage
    
    
    
    
    elif expt_no == 3:
        perecentage_of_overlap_for_T_trials = np.array(perecentage_of_overlap_for_T_trials)
        average_perecentage_of_overlap_for_T_trials = np.mean(perecentage_of_overlap_for_T_trials)
        std_dev_perecentage_of_overlap_for_T_trials = np.std(perecentage_of_overlap_for_T_trials, ddof=1)
        
        return average_perecentage_of_overlap_for_T_trials, std_dev_perecentage_of_overlap_for_T_trials
    
    
    elif expt_no == 4:
        confusion_set_Overlap_metric_for_T_trials = np.array(confusion_set_Overlap_metric_for_T_trials)
        average_confusion_set_Overlap_metric_for_T_trials = np.mean(confusion_set_Overlap_metric_for_T_trials)
        std_dev_confusion_set_Overlap_metric_for_T_trials = np.std(confusion_set_Overlap_metric_for_T_trials, ddof=1)
        
        return average_confusion_set_Overlap_metric_for_T_trials, std_dev_confusion_set_Overlap_metric_for_T_trials
        

    

# Grid search for hyperparameter tuning(k_reg, lambd) for RAPS method

In [42]:

expt_no = 4
Trials = 10
alpha = 0.01
k_reg = [1, 2, 3, 4, 5, 6, 7]
lambd = [0, 0.0001, 0.001, 0.01, 0.02, 0.05, 0.2, 0.5, 0.7, 1.0]
rand = True

dic_expt_3 = {
    'average_perecentage_of_overlap_for_T_trials' : [],
    'std_dev_perecentage_of_overlap_for_T_trials' : []
}


dic_expt_4 = {
    'average_confusion_set_Overlap_metric_for_T_trials' : [],
    'std_dev_confusion_set_Overlap_metric_for_T_trials' : []
   
}



dic_expt_1 = {
    'k_reg_and_lambd': [],
    'average_len' : [],
    'std_dev_len' : [],
    'average_coverage_gap':[],
    'std_dev_coverage_gap' : [],
    'average_coverage' : [],
    'std_dev_coverage' : []
 }




dic_expt_2 = {
    'k_reg_and_lambd': [],
    'normal_average_set_size_len' : [],
    'normal_std_dev_set_size_len' : [],
    'LG_average_set_size_len' : [],
    'LG_std_dev_set_size_len' : [],
    'HG_average_set_size_len' : [],
    'HG_std_dev_set_size_len' : [],
    
    'normal_avg_coverage_gap' : [],
    'normal_std_dev_coverage_gap' : [],
    'LG_avg_coverage_gap' : [],
    'LG_std_dev_coverage_gap' : [],
    'HG_avg_coverage_gap' : [],
    'HG_std_dev_coverage_gap' : [],
    
    'normal_average_coverage' : [],
    'normal_std_dev_coverage' : [],
    'LG_average_coverage' : [],
    'LG_std_dev_coverage' : [],
    'HG_average_coverage' : [],
    'HG_std_dev_coverage' : []
    
}



for i in range(len(k_reg)):
    for j in range(len(lambd)):
        print(f'k_reg :- {k_reg[i]}, lambd :- {lambd[j]}')
        #average_len, std_dev_len, average_coverage_gap, std_dev_coverage_gap, average_coverage, std_dev_coverage = main(expt_no, Trials, alpha, k_reg[i], lambd[j], rand)
        main(expt_no, Trials, alpha, k_reg[i], lambd[j], rand)
        if expt_no == 1:
            average_len, std_dev_len, average_coverage_gap, std_dev_coverage_gap, average_coverage, std_dev_coverage = main(expt_no, Trials, alpha, k_reg[i], lambd[j], rand)
            dic_expt_1['k_reg_and_lambd'].append((k_reg[i], lambd[j]))
            dic_expt_1['average_len'].append(average_len)
            dic_expt_1['std_dev_len'].append(std_dev_len)
            dic_expt_1['average_coverage_gap'].append(average_coverage_gap)
            dic_expt_1['std_dev_coverage_gap'].append(std_dev_coverage_gap)
            dic_expt_1['average_coverage'].append(average_coverage)
            dic_expt_1['std_dev_coverage'].append(std_dev_coverage)
        
        elif expt_no == 2:
            normal_average_set_size_len, normal_std_dev_set_size_len, LG_average_set_size_len, LG_std_dev_set_size_len, HG_average_set_size_len, HG_std_dev_set_size_len, normal_avg_coverage_gap, normal_std_dev_coverage_gap, LG_avg_coverage_gap, LG_std_dev_coverage_gap, HG_avg_coverage_gap, HG_std_dev_coverage_gap, normal_average_coverage, normal_std_dev_coverage, LG_average_coverage, LG_std_dev_coverage, HG_average_coverage, HG_std_dev_coverage = main(expt_no, Trials, alpha, k_reg[i], lambd[j], rand)
            dic_expt_2['k_reg_and_lambd'].append((k_reg[i], lambd[j]))
            
            dic_expt_2['normal_average_set_size_len'].append(normal_average_set_size_len)
            dic_expt_2['normal_std_dev_set_size_len'].append(normal_std_dev_set_size_len)
            dic_expt_2['LG_average_set_size_len'].append(LG_average_set_size_len)
            dic_expt_2['LG_std_dev_set_size_len'].append(LG_std_dev_set_size_len)
            dic_expt_2['HG_average_set_size_len'].append(HG_average_set_size_len)
            dic_expt_2['HG_std_dev_set_size_len'].append(HG_std_dev_set_size_len)
            
            dic_expt_2['normal_avg_coverage_gap'].append(normal_avg_coverage_gap)
            dic_expt_2['normal_std_dev_coverage_gap'].append(normal_std_dev_coverage_gap)
            dic_expt_2['LG_avg_coverage_gap'].append(LG_avg_coverage_gap)
            dic_expt_2['LG_std_dev_coverage_gap'].append(LG_std_dev_coverage_gap)
            dic_expt_2['HG_avg_coverage_gap'].append(HG_avg_coverage_gap)
            dic_expt_2['HG_std_dev_coverage_gap'].append(HG_std_dev_coverage_gap)
            
            dic_expt_2['normal_average_coverage'].append(normal_average_coverage)
            dic_expt_2['normal_std_dev_coverage'].append(normal_std_dev_coverage)
            dic_expt_2['LG_average_coverage'].append(LG_average_coverage)
            dic_expt_2['LG_std_dev_coverage'].append(LG_std_dev_coverage)
            dic_expt_2['HG_average_coverage'].append(HG_average_coverage)
            dic_expt_2['HG_std_dev_coverage'].append(HG_std_dev_coverage)
            
            
            
        elif expt_no == 3:
            average_perecentage_of_overlap_for_T_trials, std_dev_perecentage_of_overlap_for_T_trials = main(expt_no, Trials, alpha, k_reg[i], lambd[j], rand) 
            dic_expt_3['average_perecentage_of_overlap_for_T_trials'].append(average_perecentage_of_overlap_for_T_trials)
            dic_expt_3['std_dev_perecentage_of_overlap_for_T_trials'].append(std_dev_perecentage_of_overlap_for_T_trials)
            
            
        elif expt_no == 4:
            average_confusion_set_Overlap_metric_for_T_trials, std_dev_confusion_set_Overlap_metric_for_T_trials = main(expt_no, Trials, alpha, k_reg[i], lambd[j], rand)
            dic_expt_4['average_confusion_set_Overlap_metric_for_T_trials'].append(average_confusion_set_Overlap_metric_for_T_trials)
            dic_expt_4['std_dev_confusion_set_Overlap_metric_for_T_trials'].append(std_dev_confusion_set_Overlap_metric_for_T_trials)
            
            

k_reg :- 1, lambd :- 0
k_reg :- 1, lambd :- 0.0001
k_reg :- 1, lambd :- 0.001
k_reg :- 1, lambd :- 0.01
k_reg :- 1, lambd :- 0.02
k_reg :- 1, lambd :- 0.05
k_reg :- 1, lambd :- 0.2
k_reg :- 1, lambd :- 0.5
k_reg :- 1, lambd :- 0.7
k_reg :- 1, lambd :- 1.0
k_reg :- 2, lambd :- 0
k_reg :- 2, lambd :- 0.0001
k_reg :- 2, lambd :- 0.001
k_reg :- 2, lambd :- 0.01
k_reg :- 2, lambd :- 0.02
k_reg :- 2, lambd :- 0.05
k_reg :- 2, lambd :- 0.2
k_reg :- 2, lambd :- 0.5
k_reg :- 2, lambd :- 0.7
k_reg :- 2, lambd :- 1.0
k_reg :- 3, lambd :- 0
k_reg :- 3, lambd :- 0.0001
k_reg :- 3, lambd :- 0.001
k_reg :- 3, lambd :- 0.01
k_reg :- 3, lambd :- 0.02
k_reg :- 3, lambd :- 0.05
k_reg :- 3, lambd :- 0.2
k_reg :- 3, lambd :- 0.5
k_reg :- 3, lambd :- 0.7
k_reg :- 3, lambd :- 1.0
k_reg :- 4, lambd :- 0
k_reg :- 4, lambd :- 0.0001
k_reg :- 4, lambd :- 0.001
k_reg :- 4, lambd :- 0.01
k_reg :- 4, lambd :- 0.02
k_reg :- 4, lambd :- 0.05
k_reg :- 4, lambd :- 0.2
k_reg :- 4, lambd :- 0.5
k_reg :- 4, lambd :- 0.7
k

In [43]:
df_90 = pd.DataFrame(dic_expt_4)
df_90

,average_confusion_set_Overlap_metric_for_T_trials,std_dev_confusion_set_Overlap_metric_for_T_trials
0,33.418803,1.930830
1,29.218559,2.346914
2,25.170940,1.225757
3,22.905983,2.019231
4,21.587302,1.841244
...,...,...
65,34.065934,2.211889
66,32.515263,2.896316
67,30.384615,4.433321
68,30.873016,4.012005


In [44]:
min_index = df_90['average_confusion_set_Overlap_metric_for_T_trials'].idxmin()
min_row = df_90.loc[min_index]
min_row

average_confusion_set_Overlap_metric_for_T_trials    20.873016
std_dev_confusion_set_Overlap_metric_for_T_trials     1.839050
Name: 5, dtype: float64